reading timestamps to filter out ones we are interested in

In [1]:
import pandas as pd

timestamps_data = pd.read_csv("data_csv/timestamps_ext.csv")
target = pd.read_csv("data_csv/first_win.csv")
target.columns = ["matchId", "first_win"]
data = timestamps_data.merge(target, on="matchId")
del target
del timestamps_data

data_cleaned = data.dropna()



In [2]:
timestamps_starting = data_cleaned[data_cleaned["timeStamp"] == 0]
timestamps_20p = data_cleaned.loc[
    data_cleaned.groupby("matchId")["percentTimeStamp"].apply(
        lambda x: (x - 0.2).abs().idxmin()
    )
]
timestamps_40p = data_cleaned.loc[
    data_cleaned.groupby("matchId")["percentTimeStamp"].apply(
        lambda x: (x - 0.4).abs().idxmin()
    )
]
timestamps_60p = data_cleaned.loc[
    data_cleaned.groupby("matchId")["percentTimeStamp"].apply(
        lambda x: (x - 0.6).abs().idxmin()
    )
]
timestamps_80p = data_cleaned.loc[
    data_cleaned.groupby("matchId")["percentTimeStamp"].apply(
        lambda x: (x - 0.8).abs().idxmin()
    )
]
timestamps_100p = data_cleaned.loc[
    data_cleaned.groupby("matchId")["percentTimeStamp"].apply(
        lambda x: (x - 1.0).abs().idxmin()
    )
]


In [3]:
#make data_cleaned only include whats in timestamps_starting .. 100

data_cleaned = pd.concat(
    [timestamps_starting, timestamps_20p, timestamps_40p, timestamps_60p, timestamps_80p, timestamps_100p]
)

loading matches data to filtered timestamps

In [6]:
del data
matches_data = pd.read_csv("data_csv/matches.csv")
data_cleaned = data_cleaned.merge(matches_data, on="matchId")
print(len(data_cleaned.columns))
del matches_data
#call garbagecollector 
import gc
gc.collect()

1732


588

if matches.csv is having player ids in columns 0-9 instead of 1-10 uncomment below and paste as first line whats in output into matches.csv

In [7]:
# with open("data_csv/matches.csv", "r") as f:
#     columns = f.readline().strip().split(",")

# #replace each number in columns with number+1
# import regex as re
# regex = re.compile(r"p(\d+)_")

# columns = [regex.sub(lambda x: f"p{int(x.group(1))+1}_", col) for col in columns]

# print(",".join(columns))

In [8]:
columns_to_drop = ["gameMode", "gameName", "gameType", "gameVersion", "mapId" ]
data_cleaned.drop(columns=columns_to_drop, inplace=True)
data_cleaned.drop(columns= ["p{}_summonerId".format(i) for i in range(1, 11)], inplace=True)
data_cleaned.drop(columns= ["p{}_championName".format(i) for i in range(1, 11)], inplace=True)
data_cleaned.drop(columns= ["p{}_individualPosition".format(i) for i in range(1, 11)], inplace=True)

columns_to_drop = ["p{}_win".format(i) for i in range(1, 11)]
data_cleaned.drop(columns=["p{}_win".format(i) for i in range(1,11)], inplace=True)
data_cleaned.drop(columns=["p{}_nexusLost".format(i) for i in range(1,11)], inplace=True)
data_cleaned.drop(columns=["p{}_nexusKills".format(i) for i in range(1,11)], inplace=True)

data_cleaned.drop(columns= ["p{}_summonerName".format(i) for i in range(1, 11)], inplace=True)
data_cleaned.drop(columns= ["p{}_teamPosition".format(i) for i in range(1, 11)], inplace=True)
data_cleaned.drop(columns= ["p{}_riotIdTagline".format(i) for i in range(1, 11)], inplace=True)
data_cleaned.drop(columns= ["tournamentCode"], inplace=True)
data_cleaned.drop(columns= ["p{}_puuid".format(i) for i in range(1, 11)], inplace=True)

MemoryError: Unable to allocate 1.63 GiB for an array with shape (1510, 145302) and data type int64

In [10]:
len(data_cleaned.columns)

1626

In [12]:
#reset index
data_cleaned.reset_index(drop=True, inplace=True)
print(len(data_cleaned))

145302


In [15]:
del timestamps_100p
del timestamps_80p
del timestamps_60p
del timestamps_40p
del timestamps_20p
del timestamps_starting
gc.collect()


NameError: name 'timestamps_100p' is not defined

In [19]:
#show how many Na values are in each column and order them by the number of Na values
na_values = data_cleaned.isna().sum().sort_values(ascending=False)
print(na_values)

matchId                       0
p9_objectivesStolenAssists    0
p9_onMyWayPings               0
p8_onMyWayPings               0
p7_onMyWayPings               0
                             ..
p8_bountyLevel                0
p7_bountyLevel                0
p6_bountyLevel                0
p5_bountyLevel                0
t1_objectives_tower_kills     0
Length: 1626, dtype: int64


In [20]:
#onehot platformId
data_cleaned = pd.get_dummies(data_cleaned, columns=["platformId"], drop_first=True)

In [21]:
from sklearn.ensemble import HistGradientBoostingClassifier
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix


def evaluate_model(model, X, y):
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.3, random_state=42
    )

    model.fit(X_train, y_train)

    y_pred = model.predict(X_test)

    accuracy = accuracy_score(y_test, y_pred)

    print(f"Model: {model.__class__.__name__}")
    print(f"Accuracy: {accuracy:.4f}")


models = [
    LogisticRegression(random_state=42, n_jobs=-1),
    DecisionTreeClassifier(random_state=42),
    RandomForestClassifier(random_state=42, n_jobs=-1),
    HistGradientBoostingClassifier(random_state=42),
]

In [22]:
from sklearn.model_selection import cross_val_score


def cross_val_model(model, X, y):
    # Perform cross-validation and get accuracy scores
    scores = cross_val_score(model, X, y, cv=5, n_jobs=-1, scoring="accuracy")
    # Print the model's class name
    print(f"Model: {model.__class__.__name__}")
    # Print all cross-validation scores
    print(f"Cross-validation scores (Accuracy): {scores}")
    # Print the mean cross-validation score
    print(f"Mean cross-validation score (Accuracy): {scores.mean():.4f}\n")

In [ ]:
def test_timestamp(timestamp):
    X, y, name = timestamp
    print("timestamp at {} percent".format(name))
    for model in models:
        evaluate_model(model, X, y)

    # uncomment to also test with cross_validation - takes more time and results are similar so I left it commented out for now
    # for model in models:
    #     cross_val_model(model, X, y)

MAKING ONEHOT ENCODING OF p_championId_x is broken a bit

In [ ]:
champion_id_frame = data_cleaned[["p{}_championId_x".format(i) for i in range(1, 11)]]

#it would be cool to make from 1-5 one set of onehot encoding ex. blue_championId_1 ... blue_championId_900 
#and from 6-10 another set of onehot encoding ex. red_championId_1 ... red_championId_900

#for now below attempt created that there are multiple same name collumns ex. blue_100 blue_100 ... {5 times like for each player}.
# Combining it into one collumn after is time consuming and not sure if it even is correct

In [ ]:
raise NotImplementedError("below does not work")
#generate onehot encoding for championId that if championId occurs for any player in blue it will be 1 
# and another set of columns for red team
#print generated columns
data_cleaned =pd.get_dummies(data_cleaned, columns=["p{}_championId_x".format(i) for i in range(1,6)], prefix="blue", drop_first=True)
data_cleaned =pd.get_dummies(data_cleaned, columns=["p{}_championId_x".format(i) for i in range(6,11)], prefix="red", drop_first=True)

#there are now multiple columns with the same name, we need to merge them
#merge columns with same name as max() of the two
blue_columns = [col for col in data_cleaned.columns if col.startswith("blue_")]
for col in blue_columns:
    data_cleaned[col] = data_cleaned[blue_columns].max(axis=1)
    
red_columns = [col for col in data_cleaned.columns if col.startswith("red_")]
for col in red_columns:
    data_cleaned[col] = data_cleaned[red_columns].max(axis=1)
    

In [ ]:
models = [
    LogisticRegression(random_state=42, n_jobs=-1),
    DecisionTreeClassifier(random_state=42),
    RandomForestClassifier(random_state=42, n_jobs=-1),
    HistGradientBoostingClassifier(random_state=42),
]

In [ ]:

timestamps_starting = data_cleaned[data_cleaned["timeStamp"] == 0]
timestamps_20p = data_cleaned.loc[
    data_cleaned.groupby("matchId")["percentTimeStamp"].apply(
        lambda x: (x - 0.2).abs().idxmin()
    )
]
timestamps_40p = data_cleaned.loc[
    data_cleaned.groupby("matchId")["percentTimeStamp"].apply(
        lambda x: (x - 0.4).abs().idxmin()
    )
]
timestamps_60p = data_cleaned.loc[
    data_cleaned.groupby("matchId")["percentTimeStamp"].apply(
        lambda x: (x - 0.6).abs().idxmin()
    )
]
timestamps_80p = data_cleaned.loc[
    data_cleaned.groupby("matchId")["percentTimeStamp"].apply(
        lambda x: (x - 0.8).abs().idxmin()
    )
]
timestamps_100p = data_cleaned.loc[
    data_cleaned.groupby("matchId")["percentTimeStamp"].apply(
        lambda x: (x - 1.0).abs().idxmin()
    )
]
others = ["matchId"]

timestamps = []
X_0 = timestamps_starting.drop(columns=["first_win"] + others )
y_0 = timestamps_starting["first_win"]
timestamps.append((X_0, y_0, "0"))

X_20 = timestamps_20p.drop(columns=["first_win"] + others)
y_20 = timestamps_20p["first_win"]
timestamps.append((X_20, y_20, "20"))

X_40 = timestamps_40p.drop(columns=["first_win"] + others)
y_40 = timestamps_40p["first_win"]
timestamps.append((X_40, y_40, "40"))

X_60 = timestamps_60p.drop(columns=["first_win"] + others)
y_60 = timestamps_60p["first_win"]
timestamps.append((X_60, y_60, "60"))

X_80 = timestamps_80p.drop(columns=["first_win"] + others)
y_80 = timestamps_80p["first_win"]
timestamps.append((X_80, y_80, "80"))

X_100 = timestamps_100p.drop(columns=["first_win"] + others)
y_100 = timestamps_100p["first_win"]
timestamps.append((X_100, y_100, "100"))


In [ ]:
def print_correlation_features(X,Y):
    correlations = X.corrwith(Y).sort_values(ascending=False)

    print("\nTop 10 features most positively correlated with 'first_win':")
    print(correlations.head(10))

    print("\nTop 10 features most negatively correlated with 'first_win':")
    print(correlations.tail(10))
    
for ts, X, Y  in enumerate([(X_0, y_0), (X_20, y_20), (X_40, y_40), (X_60, y_60), (X_80, y_80), (X_100, y_100)]):
    print("Timestamp: ", ts*20)
    print_correlation_features(X, y_0)

In [ ]:
for i in range(6):
    test_timestamp(timestamps[i])